In [59]:
import pandas as pd
import numpy as np
import json
import plotly.express as px
import boto3
import config
from sqlalchemy import create_engine, text

In [3]:
# import weather dataset
weather_df = pd.read_csv("WeatherForecast 2022-09-19.csv")
print('shape : ', weather_df.shape)
weather_df.head()

shape :  (280, 35)


,Unnamed: 0,id,city,lat,lon,dt,sunrise,sunset,moonrise,moonset,...,wind_deg,wind_gust,weather_id,weather_main,weather_description,weather_icon,clouds,pop,rain,uvi
0,0,0,Mont Saint Michel,48.635954,-1.51146,1663930800,1663912423,1663956197,1663900980,1663954560,...,305,6.86,500,Rain,light rain,10d,100,0.76,1.34,0.78
1,1,0,Mont Saint Michel,48.635954,-1.51146,1664017200,1663998907,1664042469,1663991640,1664041980,...,341,7.19,500,Rain,light rain,10d,33,0.74,1.89,3.31
2,2,0,Mont Saint Michel,48.635954,-1.51146,1664103600,1664085392,1664128742,1664082360,1664129340,...,25,13.05,500,Rain,light rain,10d,89,0.73,1.83,3.32
3,3,0,Mont Saint Michel,48.635954,-1.51146,1664190000,1664171876,1664215014,1664173140,1664216640,...,299,13.00,500,Rain,light rain,10d,85,0.98,2.19,1.94
4,4,0,Mont Saint Michel,48.635954,-1.51146,1664276400,1664258361,1664301286,1664264040,1664304060,...,295,13.04,501,Rain,moderate rain,10d,98,1.00,5.91,1.58


In [4]:
weather_df.columns

Index(['Unnamed: 0', 'id', 'city', 'lat', 'lon', 'dt', 'sunrise', 'sunset',
       'moonrise', 'moonset', 'moon_phase', 'temp_day', 'temp_min', 'temp_max',
       'temp_night', 'temp_eve', 'temp_morn', 'feels_like_day',
       'feels_like_night', 'feels_like_eve', 'feels_like_morn', 'pressure',
       'humidity', 'dew_point', 'wind_speed', 'wind_deg', 'wind_gust',
       'weather_id', 'weather_main', 'weather_description', 'weather_icon',
       'clouds', 'pop', 'rain', 'uvi'],
      dtype='object')

In [5]:
weather_df['date'] = pd.to_datetime(weather_df['dt'], unit='s').astype('datetime64[ns, Europe/Paris]').dt.tz_convert('UTC')
weather_df['date'] = weather_df["date"].dt.date

C:\Users\Baptiste Cournault\AppData\Local\Temp\ipykernel_28440\3256834950.py:1: FutureWarning: Using .astype to convert from timezone-naive dtype to timezone-aware dtype is deprecated and will raise in a future version.  Use ser.dt.tz_localize instead.
  weather_df['date'] = pd.to_datetime(weather_df['dt'], unit='s').astype('datetime64[ns, Europe/Paris]').dt.tz_convert('UTC')


In [6]:
LAT_0 = 46.71109
LON_0 = 1.7191036

fig1 = px.scatter_mapbox(
    weather_df, 
    lat="lat",
    lon="lon",
    animation_frame = 'date',
    animation_group = 'city',
    zoom = 4.75,
    hover_name = 'city',
    hover_data = ['weather_description', 'pop', 'rain', 'feels_like_day', 'date'],
    center = {'lat': LAT_0, 'lon': LON_0}, 
    color='feels_like_day',
    size = 'feels_like_day',
    title = 'weather forecast for the next 7 days',
    mapbox_style='carto-positron',
    color_continuous_scale='Bluered',
    opacity = 0.7,
    height = 700,
    )

In [7]:
fig1.show()

In [8]:
df_pivot = weather_df.pivot_table(index=['id', 'city'], values=['pop', 'rain','clouds', 'feels_like_day', 'feels_like_night', 'feels_like_eve', 'feels_like_morn', 'lat', 'lon'], aggfunc='mean', sort=True)

In [9]:
df_pivot

,,clouds,feels_like_day,feels_like_eve,feels_like_morn,feels_like_night,lat,lon,pop,rain
id,city,,,,,,,,,
0,Mont Saint Michel,71.500,15.59625,13.97875,11.54875,12.43500,48.635954,-1.511460,0.79375,5.934286
1,St Malo,73.250,15.79000,14.60125,13.23000,14.06500,48.649518,-2.026041,0.78625,5.006250
2,Bayeux,74.000,14.89625,12.45250,9.92375,11.29750,49.276462,-0.702474,0.73250,5.350000
3,Le Havre,78.250,14.91875,14.21000,13.41625,13.77625,49.493898,0.107973,0.79625,6.364286
4,Rouen,86.500,14.30375,12.28875,10.16875,10.41375,49.440459,1.093966,0.79875,5.817143
5,Paris,85.625,15.27375,14.07500,11.44500,12.63875,48.858890,2.320041,0.66500,1.830000
6,Amiens,86.000,13.98375,11.85625,9.61125,10.33375,49.894171,2.295695,0.73500,4.031429
7,Lille,76.125,14.28875,12.17750,9.25875,10.97625,50.636565,3.063528,0.68375,4.630000
8,Strasbourg,91.500,14.53250,12.41125,9.02500,11.30625,48.584614,7.750713,0.76250,4.834286


In [10]:
df_pivot = df_pivot.sort_values(by=['rain', 'pop', 'clouds', 'feels_like_day'], ascending=[True, True, True, False])

In [11]:
df_pivot['weather_description'] = weather_df.groupby(['id', 'city'])['weather_description'].agg(pd.Series.mode)

In [12]:
df_pivot = df_pivot.reset_index()

In [13]:
df_pivot

,id,city,clouds,feels_like_day,feels_like_eve,feels_like_morn,feels_like_night,lat,lon,pop,rain,weather_description
0,5,Paris,85.625,15.27375,14.07500,11.44500,12.63875,48.858890,2.320041,0.66500,1.830000,light rain
1,34,La Rochelle,67.500,16.79875,16.55000,13.56750,15.72375,46.159113,-1.152043,0.64000,2.444286,light rain
2,22,Avignon,76.875,19.67500,17.50000,13.14625,15.22750,43.949249,4.805901,0.40000,3.372500,"[light rain, moderate rain]"
3,17,Gorges du Verdon,85.000,14.08625,10.43875,7.03125,8.66125,43.749656,6.328562,0.31625,3.747500,light rain
4,23,Uzes,77.625,18.85875,16.14000,12.22000,13.89375,44.012128,4.419672,0.40875,3.862500,light rain
5,6,Amiens,86.000,13.98375,11.85625,9.61125,10.33375,49.894171,2.295695,0.73500,4.031429,light rain
6,7,Lille,76.125,14.28875,12.17750,9.25875,10.97625,50.636565,3.063528,0.68375,4.630000,light rain
7,8,Strasbourg,91.500,14.53250,12.41125,9.02500,11.30625,48.584614,7.750713,0.76250,4.834286,light rain
8,1,St Malo,73.250,15.79000,14.60125,13.23000,14.06500,48.649518,-2.026041,0.78625,5.006250,light rain
9,27,Collioure,79.500,19.92500,18.82625,15.90625,17.56375,42.525050,3.083155,0.65625,5.326667,moderate rain


In [14]:
# top 5 destinations :

top_destinations = df_pivot.sort_values(by=['rain', 'pop', 'clouds', 'feels_like_day'], ascending=[True, True, True, False]).loc[0:5, 'city'].to_numpy()

top_destinations


array(['Paris', 'La Rochelle', 'Avignon', 'Gorges du Verdon', 'Uzes',
       'Amiens'], dtype=object)

In [15]:
weather_df.groupby('city')['weather_description'].agg(pd.Series.mode)

city
Aigues Mortes                                                   light rain
Aix en Provence                                                 light rain
Amiens                                                          light rain
Annecy                                                          light rain
Ariege                                                       moderate rain
Avignon                                        [light rain, moderate rain]
Bayeux                                                          light rain
Bayonne                                        [light rain, moderate rain]
Besancon                                                        light rain
Biarritz                                       [light rain, moderate rain]
Bormes les Mimosas                                           broken clouds
Carcassonne                                                     light rain
Cassis                                      [broken clouds, moderate rain]
Chateau du Haut Koen

In [16]:
df_pivot

,id,city,clouds,feels_like_day,feels_like_eve,feels_like_morn,feels_like_night,lat,lon,pop,rain,weather_description
0,5,Paris,85.625,15.27375,14.07500,11.44500,12.63875,48.858890,2.320041,0.66500,1.830000,light rain
1,34,La Rochelle,67.500,16.79875,16.55000,13.56750,15.72375,46.159113,-1.152043,0.64000,2.444286,light rain
2,22,Avignon,76.875,19.67500,17.50000,13.14625,15.22750,43.949249,4.805901,0.40000,3.372500,"[light rain, moderate rain]"
3,17,Gorges du Verdon,85.000,14.08625,10.43875,7.03125,8.66125,43.749656,6.328562,0.31625,3.747500,light rain
4,23,Uzes,77.625,18.85875,16.14000,12.22000,13.89375,44.012128,4.419672,0.40875,3.862500,light rain
5,6,Amiens,86.000,13.98375,11.85625,9.61125,10.33375,49.894171,2.295695,0.73500,4.031429,light rain
6,7,Lille,76.125,14.28875,12.17750,9.25875,10.97625,50.636565,3.063528,0.68375,4.630000,light rain
7,8,Strasbourg,91.500,14.53250,12.41125,9.02500,11.30625,48.584614,7.750713,0.76250,4.834286,light rain
8,1,St Malo,73.250,15.79000,14.60125,13.23000,14.06500,48.649518,-2.026041,0.78625,5.006250,light rain
9,27,Collioure,79.500,19.92500,18.82625,15.90625,17.56375,42.525050,3.083155,0.65625,5.326667,moderate rain


In [17]:
df_pivot = df_pivot.rename(columns={"city": "destination"})

In [18]:
df_pivot.head()

,id,destination,clouds,feels_like_day,feels_like_eve,feels_like_morn,feels_like_night,lat,lon,pop,rain,weather_description
0,5,Paris,85.625,15.27375,14.07500,11.44500,12.63875,48.858890,2.320041,0.66500,1.830000,light rain
1,34,La Rochelle,67.500,16.79875,16.55000,13.56750,15.72375,46.159113,-1.152043,0.64000,2.444286,light rain
2,22,Avignon,76.875,19.67500,17.50000,13.14625,15.22750,43.949249,4.805901,0.40000,3.372500,"[light rain, moderate rain]"
3,17,Gorges du Verdon,85.000,14.08625,10.43875,7.03125,8.66125,43.749656,6.328562,0.31625,3.747500,light rain
4,23,Uzes,77.625,18.85875,16.14000,12.22000,13.89375,44.012128,4.419672,0.40875,3.862500,light rain


In [19]:
# import hotel dataset

with open("booking_json/hotel_data.json") as json_file:
    data_json = json.load(json_file)

hotel_df = pd.DataFrame.from_dict(data_json, orient='columns')
print('shape : ', hotel_df.shape)
hotel_df.head()

shape :  (17799, 8)


,id,city,address,hotel_name,url,rating,coordinates,description
0,3,[Le Havre],"\n15 Rue Anfray, Perret, 76600 Le Havre, France\n",Le havre: superbe appartement centre ville,https://www.booking.com/hotel/fr/le-havre-supe...,8.5,"49.49238690,0.11417510",<p>Situated 1.6 km from Le Havre Beach and 800...
1,7,[Lille],"\n3, Rue Jean Bart, Lille Centre, 59000 Lille,...",Moxy Lille City,https://www.booking.com/hotel/fr/moxy-lille-ci...,8.6,"50.62783100,3.06359200","<p>Located in Lille, 300 m from Coilliot House..."
2,5,[Paris],"\n123 Rue D'Aboukir, 2nd arr., 75002 Paris, Fr...",LivinParis - Luxury 3 Bedrooms Grands-Boulevar...,https://www.booking.com/hotel/fr/the-residence...,8.0,"48.86881640,2.35037469","<p>Located in the 2nd district of Paris, just ..."
3,9,[Itterswiller],"\n28 Route des Vins, 67140 Itterswiller, France\n",Hotel Kieffer,https://www.booking.com/hotel/fr/restaurant-ki...,9.2,"48.36388675,7.43345889","<p>Located on the Alsace Wine Route, Hotel Kie..."
4,8,[Strasbourg],"\n1 Rue des Magasins, 67000 Strasbourg, France\n",City Résidence Strasbourg Centre,https://www.booking.com/hotel/fr/appart-victor...,7.0,"48.58906194,7.73879863",<p>City Résidence Strasbourg Centre is located...


In [20]:
# cleaning the dataframe

from numpy import NaN
hotel_df["city"] = hotel_df["city"].apply(lambda x: x[0] if len(x) != 0 else NaN)
hotel_df.dropna(inplace=True)

hotel_df['hotel_lat'] = hotel_df['coordinates'].apply(lambda x: float(x.split(',')[0]))
hotel_df['hotel_lon'] = hotel_df['coordinates'].apply(lambda x: float(x.split(',')[1]))

hotel_df['rating'] = hotel_df['rating'].astype(float)

In [21]:
hotel_df.head()

,id,city,address,hotel_name,url,rating,coordinates,description,hotel_lat,hotel_lon
0,3,Le Havre,"\n15 Rue Anfray, Perret, 76600 Le Havre, France\n",Le havre: superbe appartement centre ville,https://www.booking.com/hotel/fr/le-havre-supe...,8.5,"49.49238690,0.11417510",<p>Situated 1.6 km from Le Havre Beach and 800...,49.492387,0.114175
1,7,Lille,"\n3, Rue Jean Bart, Lille Centre, 59000 Lille,...",Moxy Lille City,https://www.booking.com/hotel/fr/moxy-lille-ci...,8.6,"50.62783100,3.06359200","<p>Located in Lille, 300 m from Coilliot House...",50.627831,3.063592
2,5,Paris,"\n123 Rue D'Aboukir, 2nd arr., 75002 Paris, Fr...",LivinParis - Luxury 3 Bedrooms Grands-Boulevar...,https://www.booking.com/hotel/fr/the-residence...,8.0,"48.86881640,2.35037469","<p>Located in the 2nd district of Paris, just ...",48.868816,2.350375
3,9,Itterswiller,"\n28 Route des Vins, 67140 Itterswiller, France\n",Hotel Kieffer,https://www.booking.com/hotel/fr/restaurant-ki...,9.2,"48.36388675,7.43345889","<p>Located on the Alsace Wine Route, Hotel Kie...",48.363887,7.433459
4,8,Strasbourg,"\n1 Rue des Magasins, 67000 Strasbourg, France\n",City Résidence Strasbourg Centre,https://www.booking.com/hotel/fr/appart-victor...,7.0,"48.58906194,7.73879863",<p>City Résidence Strasbourg Centre is located...,48.589062,7.738799


In [22]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16324 entries, 0 to 17798
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           16324 non-null  int64  
 1   city         16324 non-null  object 
 2   address      16324 non-null  object 
 3   hotel_name   16324 non-null  object 
 4   url          16324 non-null  object 
 5   rating       16324 non-null  float64
 6   coordinates  16324 non-null  object 
 7   description  16324 non-null  object 
 8   hotel_lat    16324 non-null  float64
 9   hotel_lon    16324 non-null  float64
dtypes: float64(3), int64(1), object(6)
memory usage: 1.4+ MB


In [23]:
df_final = df_pivot.merge(hotel_df, left_on='id', right_on='id', how='inner')

In [24]:
df_final.head()

,id,destination,clouds,feels_like_day,feels_like_eve,feels_like_morn,feels_like_night,lat,lon,pop,...,weather_description,city,address,hotel_name,url,rating,coordinates,description,hotel_lat,hotel_lon
0,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n123 Rue D'Aboukir, 2nd arr., 75002 Paris, Fr...",LivinParis - Luxury 3 Bedrooms Grands-Boulevar...,https://www.booking.com/hotel/fr/the-residence...,8.0,"48.86881640,2.35037469","<p>Located in the 2nd district of Paris, just ...",48.868816,2.350375
1,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n10, Place De La République, 11th arr., 75011...","Crowne Plaza Paris République, an IHG Hotel",https://www.booking.com/hotel/fr/holiday-inn-p...,7.8,"48.86723700,2.36544415",<p>This 19th-century building with its strikin...,48.867237,2.365444
2,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n1 rue Geoffroy Marie, 9th arr., 75009 Paris,...",Hotel Panache,https://www.booking.com/hotel/fr/hotel-panache...,8.3,"48.87322110,2.34326050","<p>Located in the 9th district of Paris, just ...",48.873221,2.343260
3,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n146 Boulevard Pereire, 17th arr., 75017 Pari...",Appart'hôtel Makom Pereire,https://www.booking.com/hotel/fr/etoile-pereir...,8.0,"48.88347596,2.29406878",<p>Appart'hôtel Makom Pereire is located 300 m...,48.883476,2.294069
4,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n12 rue de Richelieu, 1st arr., 75001 Paris, ...",Hotel Montpensier,https://www.booking.com/hotel/fr/montpensier-p...,8.0,"48.86430452,2.33623432","<p>Located in the very heart of Paris, just 40...",48.864305,2.336234


map plot n°2 : recommendation for the best hotels near the top selected destinations

In [25]:
# only keep the top 20 ranked hotels for each top 5 destinations

df_final_top = pd.DataFrame([], columns = df_final.columns)

for destination in top_destinations:
    df_final_top = pd.concat([df_final_top, df_final[df_final["destination"] == destination].nlargest(20, 'rating')])

In [26]:
df_final_top['destination'].value_counts()

Paris               20
La Rochelle         20
Avignon             20
Gorges du Verdon    20
Uzes                20
Amiens              20
Name: destination, dtype: int64

In [27]:
LAT_0 = 46.71109
LON_0 = 1.7191036

fig2 = px.scatter_mapbox(
    df_final_top, 
    lat="hotel_lat",
    lon="hotel_lon",
    zoom = 4.75,
    hover_name = 'hotel_name',
    hover_data = ['city', 'rating', 'weather_description'],
    center = {'lat': LAT_0, 'lon': LON_0}, 
    color='rating',
    size = 'rating',
    title = 'top 20 ranked hotels for the top 5 destinations',
    mapbox_style='carto-positron',
    opacity = 0.7,
    height = 700,
    )

fig2.show()

In [61]:
df_final['weather_description'] = df_final['weather_description'].apply(lambda x: (", ").join(x) if type(x).__module__ == np.__name__ else x)

In [65]:
df_final.head()

,id,destination,clouds,feels_like_day,feels_like_eve,feels_like_morn,feels_like_night,lat,lon,pop,...,weather_description,city,address,hotel_name,url,rating,coordinates,description,hotel_lat,hotel_lon
0,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n123 Rue D'Aboukir, 2nd arr., 75002 Paris, Fr...",LivinParis - Luxury 3 Bedrooms Grands-Boulevar...,https://www.booking.com/hotel/fr/the-residence...,8.0,"48.86881640,2.35037469","<p>Located in the 2nd district of Paris, just ...",48.868816,2.350375
1,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n10, Place De La République, 11th arr., 75011...","Crowne Plaza Paris République, an IHG Hotel",https://www.booking.com/hotel/fr/holiday-inn-p...,7.8,"48.86723700,2.36544415",<p>This 19th-century building with its strikin...,48.867237,2.365444
2,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n1 rue Geoffroy Marie, 9th arr., 75009 Paris,...",Hotel Panache,https://www.booking.com/hotel/fr/hotel-panache...,8.3,"48.87322110,2.34326050","<p>Located in the 9th district of Paris, just ...",48.873221,2.343260
3,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n146 Boulevard Pereire, 17th arr., 75017 Pari...",Appart'hôtel Makom Pereire,https://www.booking.com/hotel/fr/etoile-pereir...,8.0,"48.88347596,2.29406878",<p>Appart'hôtel Makom Pereire is located 300 m...,48.883476,2.294069
4,5,Paris,85.625,15.27375,14.075,11.445,12.63875,48.85889,2.320041,0.665,...,light rain,Paris,"\n12 rue de Richelieu, 1st arr., 75001 Paris, ...",Hotel Montpensier,https://www.booking.com/hotel/fr/montpensier-p...,8.0,"48.86430452,2.33623432","<p>Located in the very heart of Paris, just 40...",48.864305,2.336234


sending the final dataset as a csv file into my S3 bucket

In [180]:
aki, sak = config.ACCESS_KEY_ID, config.SECRET_ACCESS_KEY

session = boto3.Session(aws_access_key_id=aki, 
                        aws_secret_access_key=sak)

s3 = session.resource("s3")

In [182]:
bucket = s3.create_bucket(Bucket="booking-project-kayak", CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [183]:
csv = df_final.to_csv()

In [184]:
put_object = bucket.put_object(Key="booking-dataset.csv", Body=csv)

extract uploaded dataset from S3 and store it in my newly created database using AWS RDS

In [66]:
dbuser, dbpass, dbhost, dbname = config.DBUSER, config.DBPASS, config.DBHOST, config.DBNAME

engine = create_engine(f"postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}/{dbname}", echo=True)

In [67]:
# push my dataframe to the sql database
df_final.to_sql(
    "booking_hotel_table",
    engine
)

2022-10-19 14:05:23,778 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-10-19 14:05:23,779 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-19 14:05:23,820 INFO sqlalchemy.engine.Engine select current_schema()
2022-10-19 14:05:23,821 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-19 14:05:23,864 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-10-19 14:05:23,865 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-19 14:05:23,908 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-10-19 14:05:23,910 INFO sqlalchemy.engine.Engine [generated in 0.00180s] {'name': 'booking_hotel_table'}
2022-10-19 14:05:23,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-19 14:05:23,979 INFO sqlalchemy.engine.Engine 
CREATE TABLE booking_hotel_table (
	index BIGINT, 
	id BIGINT, 
	destination TEXT, 
	clouds FLOAT(53), 
	feels_like_day FL

In [72]:
# Let's query it
conn = engine.connect()
stmt = text("SELECT * FROM booking_hotel_table WHERE city = 'Riez'")
result = conn.execute(stmt)
result.fetchall()

2022-10-19 14:16:08,205 INFO sqlalchemy.engine.Engine SELECT * FROM booking_hotel_table WHERE city = 'Riez' 
2022-10-19 14:16:08,206 INFO sqlalchemy.engine.Engine [generated in 0.00071s] {}


[(5795, 17, 'Gorges du Verdon', 85.0, 14.08625, 10.43875, 7.03125, 8.66125, 43.7496562, 6.3285616, 0.31625000000000003, 3.7474999999999996, 'light rain', 'Riez', '\n37 Allée Louis Gardiol, 04500 Riez, France\n', 'Très beau appartement au centre historique de Riez', 'https://www.booking.com/hotel/fr/tres-beau-appartement-au-centre-historique-de-riez.en-gb.html?aid=304142&label=gen173bo-1DCAQoggJCF3NlYXJjaF9nb3JnZX ... (184 characters truncated) ... rooms=1&group_children=0&req_children=0&hpos=17&hapos=167&sr_order=popularity&srpvid=289f7686b2d20149&srepoch=1666025485&from=searchresults#hotelTmpl', 9.0, '43.81777265,6.09064542', '<p>Très beau appartement au centre historique de Riez is situated in Riez, 37 km from ITER / Cadarache, 35 km from Digne Golf Course, as well as 42 km from Golf du Luberon. Featuring city and mountain views, this apartment also comes with free WiFi.</p>', 43.81777265, 6.09064542),
 (5815, 17, 'Gorges du Verdon', 85.0, 14.08625, 10.43875, 7.03125, 8.66125, 43.749